# Data Wrangling: We Rate Dogs Twitter account

In [1]:
# import 
import numpy as np
import pandas as pd
import requests
import tweepy
import json
import time

## Table of contents
* Data gathering
* Data assessment
* Data cleaning
* Analysis and visualisation
* Conclusions

## Data gathering

In the first part of this project, the required data will be gathered from different sources. 

In [2]:
# create a data frame from the provided .csv-file
df_archive = pd.read_csv('twitter-archive-enhanced.csv')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
# download the provided .tsv-file programmatically
r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

In [4]:
r.status_code

200

In [5]:
# write the downloaded object into a file
with open('image-predictions.tsv', mode = 'wb') as file:
    file.write(r.content)

In [3]:
# create a data frame
df_predictions = pd.read_csv('image-predictions.tsv', sep='\t')

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [8]:
# prepare for using the Twitter API
consumer_key = 'CONSUMER KEY'
consumer_secret = 'CONSUMER SECRET'
access_token = 'ACCESS TOKEN'
access_secret = 'ACCESS SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [9]:
# Query json data for each tweet ID in the Twitter Archive of WeRateDogs
missing_ids = [] # initialise list
count = 1
start = time.time()

with open('tweet_json.txt', 'w') as outfile:
    for i in df_archive.tweet_id: # get each tweet ID
        try:
            # write the json data for each ID into a text file
            tweet = api.get_status(i, tweet_mode = 'extended')
            json.dump(tweet._json, outfile)
            outfile.write('\n') # add new line
        except tweepy.TweepError: 
            # catch error and write missing IDs into list
            missing_ids.append(i)
        print(count, ': Twitter ID ', i)
        count += 1
        
end = time.time()
print('Elapsed time: ', (end - start)/60, ' minutes')

1 : Twitter ID  892420643555336193
2 : Twitter ID  892177421306343426
3 : Twitter ID  891815181378084864
4 : Twitter ID  891689557279858688
5 : Twitter ID  891327558926688256
6 : Twitter ID  891087950875897856
7 : Twitter ID  890971913173991426
8 : Twitter ID  890729181411237888
9 : Twitter ID  890609185150312448
10 : Twitter ID  890240255349198849
11 : Twitter ID  890006608113172480
12 : Twitter ID  889880896479866881
13 : Twitter ID  889665388333682689
14 : Twitter ID  889638837579907072
15 : Twitter ID  889531135344209921
16 : Twitter ID  889278841981685760
17 : Twitter ID  888917238123831296
18 : Twitter ID  888804989199671297
19 : Twitter ID  888554962724278272
20 : Twitter ID  888202515573088257
21 : Twitter ID  888078434458587136
22 : Twitter ID  887705289381826560
23 : Twitter ID  887517139158093824
24 : Twitter ID  887473957103951883
25 : Twitter ID  887343217045368832
26 : Twitter ID  887101392804085760
27 : Twitter ID  886983233522544640
28 : Twitter ID  886736880519319552
2

226 : Twitter ID  849051919805034497
227 : Twitter ID  848690551926992896
228 : Twitter ID  848324959059550208
229 : Twitter ID  848213670039564288
230 : Twitter ID  848212111729840128
231 : Twitter ID  847978865427394560
232 : Twitter ID  847971574464610304
233 : Twitter ID  847962785489326080
234 : Twitter ID  847842811428974592
235 : Twitter ID  847617282490613760
236 : Twitter ID  847606175596138505
237 : Twitter ID  847251039262605312
238 : Twitter ID  847157206088847362
239 : Twitter ID  847116187444137987
240 : Twitter ID  846874817362120707
241 : Twitter ID  846514051647705089
242 : Twitter ID  846505985330044928
243 : Twitter ID  846153765933735936
244 : Twitter ID  846139713627017216
245 : Twitter ID  846042936437604353
246 : Twitter ID  845812042753855489
247 : Twitter ID  845677943972139009
248 : Twitter ID  845459076796616705
249 : Twitter ID  845397057150107648
250 : Twitter ID  845306882940190720
251 : Twitter ID  845098359547420673
252 : Twitter ID  844979544864018432
2

448 : Twitter ID  819015331746349057
449 : Twitter ID  819006400881917954
450 : Twitter ID  819004803107983360
451 : Twitter ID  818646164899774465
452 : Twitter ID  818627210458333184
453 : Twitter ID  818614493328580609
454 : Twitter ID  818588835076603904
455 : Twitter ID  818536468981415936
456 : Twitter ID  818307523543449600
457 : Twitter ID  818259473185828864
458 : Twitter ID  818145370475810820
459 : Twitter ID  817908911860748288
460 : Twitter ID  817827839487737858
461 : Twitter ID  817777686764523521
462 : Twitter ID  817536400337801217
463 : Twitter ID  817502432452313088
464 : Twitter ID  817423860136083457
465 : Twitter ID  817415592588222464
466 : Twitter ID  817181837579653120
467 : Twitter ID  817171292965273600
468 : Twitter ID  817120970343411712
469 : Twitter ID  817056546584727552
470 : Twitter ID  816829038950027264
471 : Twitter ID  816816676327063552
472 : Twitter ID  816697700272001025
473 : Twitter ID  816450570814898180
474 : Twitter ID  816336735214911488
4

670 : Twitter ID  790227638568808452
671 : Twitter ID  789986466051088384
672 : Twitter ID  789960241177853952
673 : Twitter ID  789903600034189313
674 : Twitter ID  789628658055020548
675 : Twitter ID  789599242079838210
676 : Twitter ID  789530877013393408
677 : Twitter ID  789314372632018944
678 : Twitter ID  789280767834746880
679 : Twitter ID  789268448748703744
680 : Twitter ID  789137962068021249
681 : Twitter ID  788908386943430656
682 : Twitter ID  788765914992902144
683 : Twitter ID  788552643979468800
684 : Twitter ID  788412144018661376
685 : Twitter ID  788178268662984705
686 : Twitter ID  788150585577050112
687 : Twitter ID  788070120937619456
688 : Twitter ID  788039637453406209
689 : Twitter ID  787810552592695296
690 : Twitter ID  787717603741622272
691 : Twitter ID  787397959788929025
692 : Twitter ID  787322443945877504
693 : Twitter ID  787111942498508800
694 : Twitter ID  786963064373534720
695 : Twitter ID  786729988674449408
696 : Twitter ID  786709082849828864
6

892 : Twitter ID  759557299618865152
893 : Twitter ID  759447681597108224
894 : Twitter ID  759446261539934208
895 : Twitter ID  759197388317847553
896 : Twitter ID  759159934323924993
897 : Twitter ID  759099523532779520
898 : Twitter ID  759047813560868866
899 : Twitter ID  758854675097526272
900 : Twitter ID  758828659922702336
901 : Twitter ID  758740312047005698
902 : Twitter ID  758474966123810816
903 : Twitter ID  758467244762497024
904 : Twitter ID  758405701903519748
905 : Twitter ID  758355060040593408
906 : Twitter ID  758099635764359168
907 : Twitter ID  758041019896193024
908 : Twitter ID  757741869644341248
909 : Twitter ID  757729163776290825
910 : Twitter ID  757725642876129280
911 : Twitter ID  757611664640446465
912 : Twitter ID  757597904299253760
913 : Twitter ID  757596066325864448
914 : Twitter ID  757400162377592832
915 : Twitter ID  757393109802180609
916 : Twitter ID  757354760399941633
917 : Twitter ID  756998049151549440
918 : Twitter ID  756939218950160384
9

...

1327 : Twitter ID  706153300320784384
1328 : Twitter ID  705975130514706432
1329 : Twitter ID  705970349788291072
1330 : Twitter ID  705898680587526145
1331 : Twitter ID  705786532653883392
1332 : Twitter ID  705591895322394625
1333 : Twitter ID  705475953783398401
1334 : Twitter ID  705442520700944385
1335 : Twitter ID  705428427625635840
1336 : Twitter ID  705239209544720384
1337 : Twitter ID  705223444686888960
1338 : Twitter ID  705102439679201280
1339 : Twitter ID  705066031337840642
1340 : Twitter ID  704871453724954624
1341 : Twitter ID  704859558691414016
1342 : Twitter ID  704847917308362754
1343 : Twitter ID  704819833553219584
1344 : Twitter ID  704761120771465216
1345 : Twitter ID  704499785726889984
1346 : Twitter ID  704491224099647488
1347 : Twitter ID  704480331685040129
1348 : Twitter ID  704364645503647744
1349 : Twitter ID  704347321748819968
1350 : Twitter ID  704134088924532736
1351 : Twitter ID  704113298707505153
1352 : Twitter ID  704054845121142784
1353 : Twitt

1543 : Twitter ID  689599056876867584
1544 : Twitter ID  689557536375177216
1545 : Twitter ID  689517482558820352
1546 : Twitter ID  689289219123089408
1547 : Twitter ID  689283819090870273
1548 : Twitter ID  689280876073582592
1549 : Twitter ID  689275259254616065
1550 : Twitter ID  689255633275777024
1551 : Twitter ID  689154315265683456
1552 : Twitter ID  689143371370250240
1553 : Twitter ID  688916208532455424
1554 : Twitter ID  688908934925697024
1555 : Twitter ID  688898160958271489
1556 : Twitter ID  688894073864884227
1557 : Twitter ID  688828561667567616
1558 : Twitter ID  688804835492233216
1559 : Twitter ID  688789766343622656
1560 : Twitter ID  688547210804498433
1561 : Twitter ID  688519176466644993
1562 : Twitter ID  688385280030670848
1563 : Twitter ID  688211956440801280
1564 : Twitter ID  688179443353796608
1565 : Twitter ID  688116655151435777
1566 : Twitter ID  688064179421470721
1567 : Twitter ID  687841446767013888
1568 : Twitter ID  687826841265172480
1569 : Twitt

1759 : Twitter ID  678755239630127104
1760 : Twitter ID  678740035362037760
1761 : Twitter ID  678708137298427904
1762 : Twitter ID  678675843183484930
1763 : Twitter ID  678643457146150913
1764 : Twitter ID  678446151570427904
1765 : Twitter ID  678424312106393600
1766 : Twitter ID  678410210315247616
1767 : Twitter ID  678399652199309312
1768 : Twitter ID  678396796259975168
1769 : Twitter ID  678389028614488064
1770 : Twitter ID  678380236862578688
1771 : Twitter ID  678341075375947776
1772 : Twitter ID  678334497360859136
1773 : Twitter ID  678278586130948096
1774 : Twitter ID  678255464182861824
1775 : Twitter ID  678023323247357953
1776 : Twitter ID  678021115718029313
1777 : Twitter ID  677961670166224897
1778 : Twitter ID  677918531514703872
1779 : Twitter ID  677895101218201600
1780 : Twitter ID  677716515794329600
1781 : Twitter ID  677700003327029250
1782 : Twitter ID  677698403548192770
1783 : Twitter ID  677687604918272002
1784 : Twitter ID  677673981332312066
1785 : Twitt

1975 : Twitter ID  673148804208660480
1976 : Twitter ID  672997845381865473
1977 : Twitter ID  672995267319328768
1978 : Twitter ID  672988786805112832
1979 : Twitter ID  672984142909456390
1980 : Twitter ID  672980819271634944
1981 : Twitter ID  672975131468300288
1982 : Twitter ID  672970152493887488
1983 : Twitter ID  672968025906282496
1984 : Twitter ID  672964561327235073
1985 : Twitter ID  672902681409806336
1986 : Twitter ID  672898206762672129
1987 : Twitter ID  672884426393653248
1988 : Twitter ID  672877615439593473
1989 : Twitter ID  672834301050937345
1990 : Twitter ID  672828477930868736
1991 : Twitter ID  672640509974827008
1992 : Twitter ID  672622327801233409
1993 : Twitter ID  672614745925664768
1994 : Twitter ID  672609152938721280
1995 : Twitter ID  672604026190569472
1996 : Twitter ID  672594978741354496
1997 : Twitter ID  672591762242805761
1998 : Twitter ID  672591271085670400
1999 : Twitter ID  672538107540070400
2000 : Twitter ID  672523490734551040
2001 : Twitt

2191 : Twitter ID  668960084974809088
2192 : Twitter ID  668955713004314625
2193 : Twitter ID  668932921458302977
2194 : Twitter ID  668902994700836864
2195 : Twitter ID  668892474547511297
2196 : Twitter ID  668872652652679168
2197 : Twitter ID  668852170888998912
2198 : Twitter ID  668826086256599040
2199 : Twitter ID  668815180734689280
2200 : Twitter ID  668779399630725120
2201 : Twitter ID  668655139528511488
2202 : Twitter ID  668645506898350081
2203 : Twitter ID  668643542311546881
2204 : Twitter ID  668641109086707712
2205 : Twitter ID  668636665813057536
2206 : Twitter ID  668633411083464705
2207 : Twitter ID  668631377374486528
2208 : Twitter ID  668627278264475648
2209 : Twitter ID  668625577880875008
2210 : Twitter ID  668623201287675904
2211 : Twitter ID  668620235289837568
2212 : Twitter ID  668614819948453888
2213 : Twitter ID  668587383441514497
2214 : Twitter ID  668567822092664832
2215 : Twitter ID  668544745690562560
2216 : Twitter ID  668542336805281792
2217 : Twitt

In [81]:
with open('missing_ids.txt', mode = 'w') as file:
    file.write('missing_ids')

In [35]:
json_list = []
with open('tweet_json.txt') as file:
    for line in file:
        json_list.append(json.loads(line))

In [39]:
json_list[0]    

{'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
 'id': 892420643555336193,
 'id_str': '892420643555336193',
 'full_text': "This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU",
 'truncated': False,
 'display_text_range': [0, 85],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 892420639486877696,
    'id_str': '892420639486877696',
    'indices': [86, 109],
    'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
    'url': 'https://t.co/MgUWQ76dJU',
    'display_url': 'pic.twitter.com/MgUWQ76dJU',
    'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
    'type': 'photo',
    'sizes': {'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'medium': {'w': 540, 'h': 528, 'resize': 'fit'},
     'small': {'w': 540, 'h': 528, 'resize': 'fit'},
     'large': {'w': 